## tool: KNNFeatureAggregator (5 баллов)

Нужно написать класс, который будет справляться с задачей генерации новых фичей по ближайшим соседям.
Принцип его работы объясним на примере. Допустим, мы находимся в каком-то пайплайне генерации признаков. Разберем псевдокод ниже:
```python
# 1
'''
    Создаем объект нашего класса - он принимает на вход информацию о том, какой будет индекс для поиска ближ. соседей.
    Далее, "обучаем" индекс, если это нужно делать (строим граф, строим ivf-табличку) и т.п.).
    После этого блока, у нас есть обученный индекс, готовый искать ближайших соседей по train_data.
'''
knn_feature_aggregator = KNNFeatureAggregator(index_info)
knn_feature_aggregator.train(train_data, index_add_info)

# 2
'''
    Считаем индексы ближайших соседей. На данном этапе мы хотим получить признаки для обучающей выборки, поэтому
        подаем в качестве query_data нашу обучалку.
    Указывам is_train=True, чтобы вернуть k ближайших соседей без учета самих себя (считая k+1 соседей + выкидывая 1 столбик).
    k указываем __МАКСИМАЛЬНОЕ_ИЗ_ТРЕБУЮЩИХСЯ_НИЖЕ__ (пока не анализируем что это значит, просто имеем в виду).

    Возвращает np.array размера (query_data.shape[0], k) с айдишниками ближ. соседей
'''
train_neighbors = knn_feature_aggregator.kneighbors(
        query_data=train_data,
        k=100,
        is_train=True,
        index_add_info=index_add_info
)

# 4 (сначала см. пункт 3 ниже)
'''
    Информацию о признаках можно подавать, например, в виде такого словаря.
    Ключи - названия результирующих колонок с новыми признаками.
    Значения - таплы из:
        1. Название оригинальной колонки, по которой агрегируемся
        2. Аггрегирующая фукнция
        3. Список из количества ближайших соседей, по которым считаем агг. функцию.
            Здесь каждое число должно быть НЕ БОЛЬШЕ k из пункта 2 (вспоминаем "__МАКСИМАЛЬНОЕ_ИЗ_ТРЕБУЮЩИХСЯ_НИЖЕ__", понимаем :)

    Пример:
        Имеем из п. 2 айдишники соседей:
        train_neighbors = array([[1, 2, 3],
                                 [2, 0, 3],
                                 [3, 1, 4],
                                 [4, 2, 1],
                                 [3, 2, 1]], dtype=uint64)

        Тогда по записи {
            ...
            'new_neighbors_age_mean': ('age', 'mean', [2, 3]),
        }

        Создадутся две новых колонки - 'new_neighbors_age_mean_2nn', 'new_neighbors_age_mean_3nn'.
        В первой будет для каждого объекта лежать средний возраст его двух ближ. соседей,
            во второй - средний возраст трех ближ. соседей.

'''
feature_info =
{
                    #  название_колонки     агг.функция               список кол-ва соседей, по которым считать агг. функцию
    'new_col_name_1': ('original_col_name_1',     'sum',                                [10, 20, 100]),
    'new_col_name_2': ('original_col_name_2',     lambda x: x.min() % 3,                [50, 80, 100])
}

# 3
'''
    Суть этого класса - генерировать новые фичи на основе ближайших соседей. Здесь мы это и делаем.
    Для этого подаем на вход айдишники соседей из обучающей выборки и саму обучающую выборку.
    Далее, подаем на вход информацию о том, "какие" признаки нам нужны, см. выше.

    Возвращает датафрейм размера (neighbor_ids.shape[0], количество_новых_фичей_по_feature_info)
'''
train_new_feature_df = knn_feature_aggregator.make_features(
    neighbor_ids=train_neighbors,
    train_data=train_data,
    feature_info=feature_info
)
train_data_with_new_features = merge(train_data, train_new_feature_df)

# 5
'''
    Для тестовой выборки пайплайн будет выглядеть аналогично, за исключением того, что is_train теперь False
'''
test_neighbors = knn_feature_aggregator.kneighbors(
        query_data=test_data,
        k=100,
        is_train=False,
        index_add_info=index_add_info
)
test_new_feature_df = knn_feature_aggregator.make_features(
    neighbor_ids=test_neighbors,
    train_data=train_data,
    feature_info=feature_info
)
test_data_with_new_features = merge(test_data, test_new_feature_df)

```

### Задание:
Написать класс, который реализует все, что описано выше, в частности:

**\_\_init\_\_**
- вы сами решаете, какой будет индекс, будет ли он фиксирован и т.п.

**train**
- обучающую выборку не нужно сохранять в объект класса в целях экономии памяти
- если вам нужно разбить `train` на `train` и `add_items`,
      чтобы поддерживать обучение индекса на репрезентативном сабсэмпле, можете это сделать
- аргумент train_data - не обязательно выборка со всеми признаками.
      Вы хотите подавать сюда то подмножество признаков, по которому будете искать соседей
      (соответственно, нужно подавать уже приведенные к однородному виду данные)

**kneighbors**
- обязательна поддержка флажка is_train с описанным выше функционалом
- аргумент query_data - см. замечание к аргументу train_data из метода train выше

**make_features**
- обработайте отдельно случай, когда вы в качестве ближайших соседей подаете единственное число.
      Не нужно извне подавать список из одного числа, обработка должна быть внутри

**Эффективность**

Все должно быть реализовано эффективно. В том числе:
- без цикла for по всем объектам train_data/query_data
- без pd.DataFrame.apply
- можно использовать np.apply_along_axis (работает в ~5 раз быстрее, чем pandas)

**Пример**

Нужно привести пример работы вашего класса, запустив ячейки в блоке "Пример" ниже.
Не удаляйте авторский пример!

**Вопросы**

Нужно ответить на вопросы в блоке "Вопросы" ниже

**Note:** feature_info можете реализовать в любом виде, но описанный выше способ хорош тем,
      что его легко привести в удобный для дальнейшей работы вид:

In [9]:
import pandas as pd

feature_info = {
                    #  название_колонки     агг.функция               список кол-ва соседей, по которым считать агг. функцию
    'new_col_name_1': ('original_col_name_1',     lambda x: x.sum(),                                [10, 20, 100]),
    'new_col_name_2': ('original_col_name_1',     lambda x: x.mean(),                                [11, 21, 101]),
    'new_col_name_3': ('original_col_name_2',     lambda x: x.min() % 3,                [50, 80, 100])
}
pd.DataFrame(feature_info, index=['col_name', 'func', 'k']).T.explode('k').reset_index(names='new_col')

,new_col,col_name,func,k
0,new_col_name_1,original_col_name_1,<function <lambda> at 0x786c3064f5b0>,10
1,new_col_name_1,original_col_name_1,<function <lambda> at 0x786c3064f5b0>,20
2,new_col_name_1,original_col_name_1,<function <lambda> at 0x786c3064f5b0>,100
3,new_col_name_2,original_col_name_1,<function <lambda> at 0x786bfdc19bd0>,11
4,new_col_name_2,original_col_name_1,<function <lambda> at 0x786bfdc19bd0>,21
5,new_col_name_2,original_col_name_1,<function <lambda> at 0x786bfdc19bd0>,101
6,new_col_name_3,original_col_name_2,<function <lambda> at 0x786bf608d870>,50
7,new_col_name_3,original_col_name_2,<function <lambda> at 0x786bf608d870>,80
8,new_col_name_3,original_col_name_2,<function <lambda> at 0x786bf608d870>,100


In [47]:
class KNNFeatureAggregator:
    def __init__(self, index_info):
        self.index_info = index_info

    def train(self, train_data, index_add_info=None):
        train_data_np = train_data.to_numpy().astype(np.float32)

        fixed_params = self.index_info['fixed_params']
        build_func = self.index_info['build_func']

        self.index, build_time = build_func(train_data_np, **fixed_params)
        print(f"Index is finally built. Total index building time : {build_time} s")

    def kneighbors(self, query_data, index_add_info, k=100, is_train=True):
        query_data_np = query_data.to_numpy().astype(np.float32)

        search_param = index_add_info['search_param'][1]
        search_func = index_add_info['search_func']
        if is_train:
            _, labels, _ = search_func(self.index, query_data_np, k+1, search_param)
            mask = labels != np.arange(labels.shape[0])[:, np.newaxis]
            labels = labels[mask].reshape(query_data.shape[0], k)
        else:
            _, labels, _ = search_func(self.index, query_data_np, k, search_param)

        return labels

    def make_features(self, neighbor_ids, train_data, feature_info):
        new_features = pd.DataFrame()

        for new_col_name, params in feature_info.items():
            orig_col_name, agg_func, nn = params
            if isinstance(nn, int):
                nn_ids = neighbor_ids[:,:nn].astype(np.int64)
                nn_data = np.take(train_data.loc[:, orig_col_name].to_numpy(), nn_ids)
                new_column = np.apply_along_axis(agg_func, axis=1, arr=nn_data)
                new_features[new_col_name] = new_column
            elif isinstance(nn, list):
                for n in nn:
                    new_name = new_col_name + "_" + str(n) + "nn"
                    nn_ids = neighbor_ids[:,:n].astype(np.int64)
                    nn_data = np.take(train_data.loc[:, orig_col_name].to_numpy(), nn_ids)
                    new_column = np.apply_along_axis(agg_func, axis=1, arr=nn_data)
                    new_features[new_name] = new_column
            else:
                print("Third parameter must be a single integer or a list!")

        return new_features

### Пример

Ваш:

In [10]:
#!pip install --upgrade faiss-cpu hnswlib

In [51]:
from time import time as tm
import faiss
import hnswlib


def timer(func):
    '''
    декоратор, замеряющий время работы функции
    '''
    def wrapper(*args, **kwargs):
        start_time = tm()
        result = func(*args, **kwargs)
        end_time = tm() - start_time
        if isinstance(result, tuple):
            return *result, end_time
        return result, end_time
    return wrapper

@timer
def build_hnsw(build_data, **fixed_params):
    dim = fixed_params['dim']
    space = fixed_params['space']
    M = fixed_params['M']
    ef_construction = fixed_params['ef_construction']

    # Declaring index
    index = hnswlib.Index(space = space, dim = dim)

    # Initializing index - the maximum number of elements should be known beforehand
    num_elements = build_data.shape[0]
    index.init_index(max_elements = num_elements, ef_construction = ef_construction, M = M)

    # Element insertion (can be called several times):
    index.add_items(build_data)

    return index

@timer
def search_hnsw(index, query_data, k_neighbors, efSearch):
    # Controlling the recall by setting ef:
    index.set_ef(efSearch)

    # Query dataset, k - number of closest elements (returns 2 numpy arrays)
    labels, distances = index.knn_query(query_data, k_neighbors)

    return distances, labels

In [52]:
index_info = {
    'fixed_params': {
        'dim': 2,
        'space': 'l2',
        'M': 32,
        'ef_construction': 256
    },
    'build_func': build_hnsw
}

In [53]:
index_add_info = {
    'search_param': ('efSearch', 256),
    'search_func': search_hnsw
}

In [54]:
train_data = pd.DataFrame({
    'a': [1, 2, 3, 4, 5],
    'b': [10, 19, 27, 34, 40]
})
agg = KNNFeatureAggregator(index_info)
agg.train(train_data)
neighbor_ids = agg.kneighbors(train_data, is_train=True, k=3, index_add_info=index_add_info)
neighbor_ids # у вас индексы ближ. соседей могут отличаться

Index is finally built. Total index building time : 0.0013282299041748047 s


array([[1, 2, 3],
       [2, 0, 3],
       [3, 1, 4],
       [4, 2, 1],
       [3, 2, 1]], dtype=uint64)

In [55]:
X = agg.make_features(neighbor_ids, train_data, feature_info={
    'a_sum': ('a', lambda x: x.sum(), [2, 3]),
    'b_whatever': ('b', lambda x: x.min(), 2),
})
X

,a_sum_2nn,a_sum_3nn,b_whatever
0,5,9,19
1,4,8,10
2,6,11,19
3,8,10,27
4,7,9,27


Авторский:

In [ ]:
train_data = pd.DataFrame({
    'a': [1, 2, 3, 4, 5],
    'b': [10, 19, 27, 34, 40]
})
agg = KNNFeatureAgg(dim=2, metric='l2') # у автора: hnsw index
agg.train(train_data)
neighbor_ids = agg.kneighbors(train_data, is_train=True, k=3)
neighbor_ids # у вас индексы ближ. соседей могут отличаться

array([[1, 2, 3],
       [2, 0, 3],
       [3, 1, 4],
       [4, 2, 1],
       [3, 2, 1]], dtype=uint64)

In [ ]:
X = agg.make_features(neighbor_ids, feature_info={
    'a_sum': ('a', lambda x: x.sum(), [2, 3]),
    'b_whatever': ('b', lambda x: x.min(), 2),
})
X

,a_sum_2nn,b_whatever_2nn,a_sum_3nn
0,5,19,9
1,4,10,8
2,6,19,11
3,8,27,10
4,7,27,9


### Вопросы

1) Какой / какие индекс[-ы] вы решили использовать для этой задачи и почему?

2) Какие недостатки / потенциальные зоны для улучшения у вашей текущей реализации?

1) В данном ноутбуке я еще не могу точно сказать какой индекс лучше всего подходит к данной задаче, потому что пока что я использовал в качестве индекса можно сказать "рандомный" алгоритм построения индекса в том смысле, что он никак не основан на реальных данных, возможно, какой-то другой индекс на данной задаче покажет лучший результат, но для этого необходимо провести какие-то эксперименты (я проведу их во втором задании). Ну а так навскидку, я брал, конечно же не прям рандом рандом, а посмотрел семинар 12 и взял на свой взгляд "оптимальный" алгоритм по соотношению полнота-время построения индекса.

P.S. После ознакомления с задачей более детально, я решил использовать LSH, потому что он поддерживает кастомную метрику.

2) Возможно, на игрушечных данных пока мне хватает и функционала и параметров (например, я никак не использую в своей реализации параметр 'index_add_info' в методе train). По эффективности вычислений - вроде бы все нормально, я использовал только функции numpy и не итерировался по строкам датафрейма и не хранил его в памяти объекта. Улучшение вижу именно в подборе оптимально индекса под данные, ну и, соответсвенно, в дописывании build и search для других индексов. Возможно я как-то не так понял вопрос, но вроде все расписал, что думаю.

Сори за лонгрид :(

Эту реализацию я добавлю в файл utils.py